# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f00acae1850>
├── 'a' --> tensor([[-1.2664, -0.4659,  0.2130],
│                   [-1.0781, -0.0200,  0.4618]])
└── 'x' --> <FastTreeValue 0x7f00acae1f70>
    └── 'c' --> tensor([[-0.5143, -2.0262, -0.3854,  0.6029],
                        [ 0.7358, -1.8799,  0.1225, -1.2607],
                        [-0.3302, -0.8050, -2.1470,  0.9154]])

In [4]:
t.a

tensor([[-1.2664, -0.4659,  0.2130],
        [-1.0781, -0.0200,  0.4618]])

In [5]:
%timeit t.a

62.5 ns ± 0.0816 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f00acae1850>
├── 'a' --> tensor([[ 0.8505, -0.3396,  0.4772],
│                   [ 0.0310, -0.9491,  0.1629]])
└── 'x' --> <FastTreeValue 0x7f00acae1f70>
    └── 'c' --> tensor([[-0.5143, -2.0262, -0.3854,  0.6029],
                        [ 0.7358, -1.8799,  0.1225, -1.2607],
                        [-0.3302, -0.8050, -2.1470,  0.9154]])

In [7]:
%timeit t.a = new_value

66.2 ns ± 0.153 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.2664, -0.4659,  0.2130],
               [-1.0781, -0.0200,  0.4618]]),
    x: Batch(
           c: tensor([[-0.5143, -2.0262, -0.3854,  0.6029],
                      [ 0.7358, -1.8799,  0.1225, -1.2607],
                      [-0.3302, -0.8050, -2.1470,  0.9154]]),
       ),
)

In [10]:
b.a

tensor([[-1.2664, -0.4659,  0.2130],
        [-1.0781, -0.0200,  0.4618]])

In [11]:
%timeit b.a

61.5 ns ± 0.0383 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4565, -0.5508,  0.2961],
               [ 0.4107, -3.9786,  0.9531]]),
    x: Batch(
           c: tensor([[-0.5143, -2.0262, -0.3854,  0.6029],
                      [ 0.7358, -1.8799,  0.1225, -1.2607],
                      [-0.3302, -0.8050, -2.1470,  0.9154]]),
       ),
)

In [13]:
%timeit b.a = new_value

500 ns ± 0.107 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

829 ns ± 0.628 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 7.95 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7efff5d85fd0>
├── 'a' --> tensor([[[-1.2664, -0.4659,  0.2130],
│                    [-1.0781, -0.0200,  0.4618]],
│           
│                   [[-1.2664, -0.4659,  0.2130],
│                    [-1.0781, -0.0200,  0.4618]],
│           
│                   [[-1.2664, -0.4659,  0.2130],
│                    [-1.0781, -0.0200,  0.4618]],
│           
│                   [[-1.2664, -0.4659,  0.2130],
│                    [-1.0781, -0.0200,  0.4618]],
│           
│                   [[-1.2664, -0.4659,  0.2130],
│                    [-1.0781, -0.0200,  0.4618]],
│           
│                   [[-1.2664, -0.4659,  0.2130],
│                    [-1.0781, -0.0200,  0.4618]],
│           
│                   [[-1.2664, -0.4659,  0.2130],
│                    [-1.0781, -0.0200,  0.4618]],
│           
│                   [[-1.2664, -0.4659,  0.2130],
│                    [-1.0781, -0.0200,  0.4618]]])
└── 'x' --> <FastTreeValue 0x7efff5d85a00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 58.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7efffc9feca0>
├── 'a' --> tensor([[-1.2664, -0.4659,  0.2130],
│                   [-1.0781, -0.0200,  0.4618],
│                   [-1.2664, -0.4659,  0.2130],
│                   [-1.0781, -0.0200,  0.4618],
│                   [-1.2664, -0.4659,  0.2130],
│                   [-1.0781, -0.0200,  0.4618],
│                   [-1.2664, -0.4659,  0.2130],
│                   [-1.0781, -0.0200,  0.4618],
│                   [-1.2664, -0.4659,  0.2130],
│                   [-1.0781, -0.0200,  0.4618],
│                   [-1.2664, -0.4659,  0.2130],
│                   [-1.0781, -0.0200,  0.4618],
│                   [-1.2664, -0.4659,  0.2130],
│                   [-1.0781, -0.0200,  0.4618],
│                   [-1.2664, -0.4659,  0.2130],
│                   [-1.0781, -0.0200,  0.4618]])
└── 'x' --> <FastTreeValue 0x7efff5d85940>
    └── 'c' --> tensor([[-0.5143, -2.0262, -0.3854,  0.6029],
                        [ 0.7358, -1.8799,  0.1225, -1.2607],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 41.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58 µs ± 68.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.5143, -2.0262, -0.3854,  0.6029],
                       [ 0.7358, -1.8799,  0.1225, -1.2607],
                       [-0.3302, -0.8050, -2.1470,  0.9154]],
              
                      [[-0.5143, -2.0262, -0.3854,  0.6029],
                       [ 0.7358, -1.8799,  0.1225, -1.2607],
                       [-0.3302, -0.8050, -2.1470,  0.9154]],
              
                      [[-0.5143, -2.0262, -0.3854,  0.6029],
                       [ 0.7358, -1.8799,  0.1225, -1.2607],
                       [-0.3302, -0.8050, -2.1470,  0.9154]],
              
                      [[-0.5143, -2.0262, -0.3854,  0.6029],
                       [ 0.7358, -1.8799,  0.1225, -1.2607],
                       [-0.3302, -0.8050, -2.1470,  0.9154]],
              
                      [[-0.5143, -2.0262, -0.3854,  0.6029],
                       [ 0.7358, -1.8799,  0.1225, -1.2607],
                       [-0.3302, -0.8050, -2.1470,  0.9154]],

In [26]:
%timeit Batch.stack(batches)

80.7 µs ± 1.99 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.5143, -2.0262, -0.3854,  0.6029],
                      [ 0.7358, -1.8799,  0.1225, -1.2607],
                      [-0.3302, -0.8050, -2.1470,  0.9154],
                      [-0.5143, -2.0262, -0.3854,  0.6029],
                      [ 0.7358, -1.8799,  0.1225, -1.2607],
                      [-0.3302, -0.8050, -2.1470,  0.9154],
                      [-0.5143, -2.0262, -0.3854,  0.6029],
                      [ 0.7358, -1.8799,  0.1225, -1.2607],
                      [-0.3302, -0.8050, -2.1470,  0.9154],
                      [-0.5143, -2.0262, -0.3854,  0.6029],
                      [ 0.7358, -1.8799,  0.1225, -1.2607],
                      [-0.3302, -0.8050, -2.1470,  0.9154],
                      [-0.5143, -2.0262, -0.3854,  0.6029],
                      [ 0.7358, -1.8799,  0.1225, -1.2607],
                      [-0.3302, -0.8050, -2.1470,  0.9154],
                      [-0.5143, -2.0262, -0.3854,  0.6029],
                   

In [28]:
%timeit Batch.cat(batches)

146 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

325 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
